# Water Quality CSV Transects joined by TIME/DATE 

In [1]:
import os
import pandas
from scripts import wqt_timestamp_match
from datetime import datetime

In [2]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))

# example gain file
wq_file = os.path.join(wd, "scripts", "tests", "testfiles", "Arc_040413\\Arc_040413_WQ\\Arc_040413_wqp_cc1.csv")


print(wq_file)

C:\Users\Andy\Documents\arcproject-wq-processing\scripts\tests\testfiles\Arc_040413\Arc_040413_WQ\Arc_040413_wqp_cc1.csv


### Import CSV file from the Sonde as Pandas Dataframe

In [36]:
wq_gain = wqt_timestamp_match.wq_from_file(wq_file)
print(wq_gain.head())

            Date_Time   Temp    pH SpCond DO_PCT    DO  DEP25  PAR RPAR  \
1 2013-04-04 11:49:52  17.24  7.82    523   79.3  6.32  -0.06  477  753   
2 2013-04-04 11:49:53  17.24  7.82    523   77.7  6.20  -0.04  482  759   
3 2013-04-04 11:49:54  17.24  7.82    523   77.7  6.20  -0.04  482  759   
4 2013-04-04 11:49:55  17.24  7.82    523   76.3  6.09  -0.03  469  766   
5 2013-04-04 11:49:56  17.24  7.82    523   76.3  6.09  -0.03  469  766   

  TurbSC   CHL CHL_VOLTS               WQ_SOURCE  
1   19.0  3.17   0.03520  Arc_040413_wqp_cc1.csv  
2   16.1  3.67   0.03882  Arc_040413_wqp_cc1.csv  
3   16.1  3.67   0.03882  Arc_040413_wqp_cc1.csv  
4   16.4  4.04   0.04266  Arc_040413_wqp_cc1.csv  
5   16.4  4.04   0.04266  Arc_040413_wqp_cc1.csv  


In [37]:
# Convert water quality columns to dtype floats

# if field is dtype string convert to numeric
for column in list(wq_gain.columns.values): 
    if wq_gain[column].dtype == object:
        wq_gain[column] = wq_gain[column].convert_objects(convert_numeric=True)

#wq_gain[field_list] = wq_gain[field_list].astype(float)


Date_Time
datetime64[ns]
Temp
object
convert
pH
object
convert
SpCond
object
convert
DO_PCT
object
convert
DO
object
convert
DEP25
object
convert
PAR
object
convert
RPAR
object
convert
TurbSC
object
convert
CHL
object
convert
CHL_VOLTS
object
convert
WQ_SOURCE
object
convert


### average top 1m of gain vertical profile

In [38]:
# Create variable with TRUE if depth is greater than 0 and less than 1
depth1m = (wq_gain['DEP25'] > 0)  & (wq_gain['DEP25'] < 1 ) 

# Select all cases where depth1m is TRUE
wq_gain_1m = wq_gain[depth1m]

print(wq_gain_1m)


             Date_Time   Temp    pH  SpCond  DO_PCT    DO  DEP25  PAR  RPAR  \
7  2013-04-04 11:49:58  17.27  7.83     523    75.5  6.02   0.07  470   772   
8  2013-04-04 11:49:59  17.27  7.83     523    75.5  6.02   0.07  530   780   
9  2013-04-04 11:50:00  17.28  7.82     523    74.4  5.93   0.16  399   786   
10 2013-04-04 11:50:01  17.28  7.82     523    74.4  5.93   0.16  399   786   
11 2013-04-04 11:50:02  17.29  7.81     523    74.3  5.92   0.24  199   795   
12 2013-04-04 11:50:03  17.29  7.81     523    74.3  5.92   0.24  199   795   
13 2013-04-04 11:50:04  17.28  7.82     523    74.3  5.92   0.45  143   802   
14 2013-04-04 11:50:05  17.28  7.82     523    74.3  5.92   0.45  143   802   
15 2013-04-04 11:50:06  17.26  7.81     523    74.3  5.93   0.63  105   805   
16 2013-04-04 11:50:07  17.26  7.81     523    74.3  5.93   0.63  105   805   
17 2013-04-04 11:50:08  17.26  7.82     523    74.4  5.93   0.77   72   816   
18 2013-04-04 11:50:09  17.26  7.82     523    74.4 

In [43]:
# get mean for depth1m
avg_1m = wq_gain_1m.mean()

print(avg_1m)

# convert series to dataframe
avg_1m_df = avg_1m.to_frame().transpose()
print(avg_1m_df)


Temp          17.263500
pH             7.814500
SpCond       523.350000
DO_PCT        73.715000
DO             5.878500
DEP25          0.342500
PAR          265.400000
RPAR         790.850000
TurbSC        25.415000
CHL            4.804000
CHL_VOLTS      0.047311
dtype: float64
      Temp      pH  SpCond  DO_PCT      DO   DEP25    PAR    RPAR  TurbSC  \
0  17.2635  7.8145  523.35  73.715  5.8785  0.3425  265.4  790.85  25.415   

     CHL  CHL_VOLTS  
0  4.804   0.047311  


### Add gain setting and site to new column

In [80]:
# get date from the first row of the dataframe
date = wq_gain["Date_Time"][1]

print(date)
length = len(wq_gain.index)
end = wq_gain["Date_Time"][length]

print(end)

2013-04-04 11:49:52
2013-04-04 11:50:39


In [81]:
# add gain and site columns to results dataframe

avg_1m_df['Date_Time'] = date
avg_1m_df['End_Time'] = end
avg_1m_df['Site'] = "CC1"
avg_1m_df['Gain'] = "g0"

avg_1m_df


,Temp,pH,SpCond,DO_PCT,DO,DEP25,PAR,RPAR,TurbSC,CHL,CHL_VOLTS,Gain,Site,Date_Time,End_Time
0,17.2635,7.8145,523.35,73.715,5.8785,0.3425,265.4,790.85,25.415,4.804,0.047311,g0,CC1,2013-04-04 11:49:52,2013-04-04 11:50:39
